In [1]:
 !pip install transformers

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 KB 5.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx


In [5]:
df=pd.read_excel('/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6405 entries, 0 to 6404
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6405 non-null   int64 
 1   ID          6404 non-null   object
 2   Text        6404 non-null   object
 3   Sentiment   6405 non-null   object
 4   Label       6405 non-null   int64 
 5   Token       6405 non-null   object
 6   Token_ID    6405 non-null   object
dtypes: int64(2), object(5)
memory usage: 350.4+ KB


In [6]:
pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 38.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 43.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 57.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) 

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [8]:
import ktrain
from ktrain import text

In [9]:
train=df[:int(.8*len(df))]
test=df[int(.8*len(df)):int(.9*len(df))]
validation=df[int(.9*len(df)):]
x_train=train['Text'].tolist()
y_train=train['Label'].tolist()
x_validation=validation['Text'].tolist()
y_validation=validation['Label'].tolist()
x_test=test['Text'].tolist()
y_test=test['Sentiment'].tolist()

In [10]:
x_train= np.array(x_train)
x_validation= np.array(x_validation)
x_test= np.array(x_test)


In [11]:
categories = ['angry','disgust','fear','happy','sad','surpise']

In [12]:
model_name = 'Kowsher/bangla-bert'
trans = text.Transformer(model_name,maxlen=50,class_names=categories)

Downloading:   0%|          | 0.00/638 [00:00<?, ?B/s]

In [13]:
train = trans.preprocess_train(x_train,y_train)
valid = trans.preprocess_test(x_validation,y_validation)

preprocessing train...
language: bn
train sequence lengths:
	mean : 16
	95percentile : 44
	99percentile : 102


Downloading:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 16
	95percentile : 49
	99percentile : 96


In [14]:
model = trans.get_classifier()

404 Client Error: Not Found for url: https://huggingface.co/Kowsher/bangla-bert/resolve/main/tf_model.h5
/opt/conda/lib/python3.7/site-packages/ktrain/text/preprocessor.py:1070: UserWarning: Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. You will need PyTorch installed for this.
  warnings.warn('Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. ' +\


Downloading:   0%|          | 0.00/658M [00:00<?, ?B/s]

In [15]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=12)

In [16]:
from sklearn.utils import class_weight
class_weight=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(y_train),y=y_train)
weight={i:class_weight[i] for i in range(6)}

In [17]:
learner.fit_onecycle(2e-5,20,class_weight=weight) 



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/20
427/427 [==============================] - 1614s 4s/step - loss: 1.7514 - accuracy: 0.2584 - val_loss: 1.6133 - val_accuracy: 0.3682
Epoch 2/20
427/427 [==============================] - 1574s 4s/step - loss: 1.5756 - accuracy: 0.4270 - val_loss: 1.4202 - val_accuracy: 0.4555
Epoch 3/20
427/427 [==============================] - 1577s 4s/step - loss: 1.4150 - accuracy: 0.4955 - val_loss: 1.3961 - val_accuracy: 0.4493
Epoch 4/20
427/427 [==============================] - 1584s 4s/step - loss: 1.2093 - accuracy: 0.5824 - val_loss: 1.4125 - val_accuracy: 0.4618
Epoch 5/20
427/427 [==============================] - 1577s 4s/step - loss: 0.9188 - accuracy: 0.6815 - val_loss: 1.4693 - val_accuracy: 0.5023
Epoch 6/20
427/427 [==============================] - 1571s 4s/step - loss: 0.5763 - accuracy: 0.7990 - val_loss: 1.8061 - val_accuracy: 0.4805
Epoch 7/20
427/427 [==============================] - 1578s 4s/step - los

In [18]:
learner.validate(class_names=categories)

              precision    recall  f1-score   support

       angry       0.60      0.65      0.62       137
     disgust       0.25      0.19      0.22        72
        fear       0.19      0.23      0.21        30
       happy       0.75      0.75      0.75       195
         sad       0.42      0.47      0.44       132
     surpise       0.32      0.24      0.27        75

    accuracy                           0.52       641
   macro avg       0.42      0.42      0.42       641
weighted avg       0.52      0.52      0.52       641



array([[ 89,  11,   5,   4,  17,  11],
       [ 21,  14,   2,   7,  21,   7],
       [  0,   4,   7,   5,   9,   5],
       [ 14,   5,   7, 146,  17,   6],
       [ 13,  16,  11,  20,  62,  10],
       [ 11,   7,   4,  12,  23,  18]])

In [19]:
predictor = ktrain.get_predictor(learner.model, preproc=trans)

In [20]:
y_pred = predictor.predict(x_test)
print(y_pred)

['angry', 'disgust', 'angry', 'sad', 'sad', 'sad', 'sad', 'angry', 'happy', 'happy', 'disgust', 'angry', 'happy', 'angry', 'angry', 'sad', 'happy', 'happy', 'disgust', 'happy', 'disgust', 'surpise', 'disgust', 'angry', 'angry', 'happy', 'happy', 'sad', 'angry', 'fear', 'happy', 'surpise', 'fear', 'happy', 'sad', 'angry', 'disgust', 'angry', 'happy', 'happy', 'sad', 'angry', 'sad', 'happy', 'angry', 'surpise', 'angry', 'surpise', 'surpise', 'angry', 'angry', 'angry', 'sad', 'happy', 'disgust', 'sad', 'sad', 'disgust', 'angry', 'surpise', 'surpise', 'fear', 'angry', 'happy', 'disgust', 'sad', 'angry', 'sad', 'surpise', 'fear', 'happy', 'angry', 'sad', 'happy', 'surpise', 'disgust', 'disgust', 'angry', 'fear', 'sad', 'sad', 'angry', 'angry', 'sad', 'angry', 'happy', 'disgust', 'sad', 'disgust', 'happy', 'disgust', 'happy', 'happy', 'happy', 'surpise', 'sad', 'fear', 'angry', 'surpise', 'sad', 'happy', 'angry', 'disgust', 'disgust', 'surpise', 'surpise', 'happy', 'happy', 'surpise', 'surpi

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       angry       0.54      0.56      0.55       144
     disgust       0.19      0.18      0.19        66
        fear       0.11      0.12      0.12        32
       happy       0.76      0.71      0.73       187
         sad       0.43      0.42      0.42       148
     surpise       0.23      0.29      0.26        63

    accuracy                           0.48       640
   macro avg       0.38      0.38      0.38       640
weighted avg       0.49      0.48      0.49       640

